## Chapter 3 평가

앞에서는 모델 예측 성능의 평가를 위해 정확도(Accuracy)를 이용했다. 하지만 이 밖에도 여러 가지 방법으로 예측 성능을 평가할 수 있다. <br/> 성능 평가 모델은 일반적으로 모델이 분류냐 회귀냐에 따라 다르다. <br/> 분류의 성능 평가 지표는 다음과 같다.
- 정확도(Accuracy)
- 오차행렬(Confusion Matrix0
- 정밀도(Precision)
- 재현율(Recall)
- F1 스코어
- ROC AUC

분류의 경우 결정 클래스 값 종류의 유형에 따라 이진 분류와 선택 분류로 나뉠 수 있다. 위의 지표들은 모두 적용되는 지표이지만  특히 이진 분류에서 더욱 중요하게 강조하는 지표이다.

### 1. 정확도(Accuracy)

정확도는 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표이며 전체 예측 데이터 건수에서 예측 결과가 동일한 데이터 건수의 비율이다. 직관적으로 모델 예측 성능을 나타낸다. 하지만 불균형한 데이터에 의한 정확도의 착시 현상에 속지 않도록, 특히 이진 판단일 경우 주의해야 한다.

### 2. 오차 행렬

이진 분류에서 성능 지표로 잘 활용되는 오차 행렬(confusion matrix, 혼동행렬)은 학습된 분류 모델이 예측을 수행하면성 얼마나 핫갈려하는지도 함께 보여주는 지표이다.

![오차 행렬](./images/confusionMatrix.png)

위의 TN, FP, FN, TP로 각 4분면을 채우고 이를 다양하게 결합해 분류 모델 성능의 오류가 어떠한 모습으로 발생하는 지 알 수 있다. <br/> 사이킷런 역시 이를 위한 API를 제공한다.

```python
from sklearn.metricx import confusion_matrix

confusion_matrix(y_test, pred)
```

이렇게 하면 반환값으로 각 행렬에 위치하는 값의 수가 2차원 리스트로 반환된다.

### 3. 정밀도와 재현율

정밀도와 재현율은 Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표이다.
- 정밀도 = TP / (FP + TP)
- 재현율 = TP / (FN + TP)

정밀도는 예측을 긍정으로 한 대상 중 실제 값이 긍정으로 일치한 데이터의 비율이다. <br/> 재현율은 실제 값이 긍정인 대상 중 예측과 실제 값이 긍정으로 일치한 데이터의 비율이다. <br/> 각각 업무 특성에 따라 특정 지표가 더 중요한 지표로 간주될 수 있다.

- 재현율이 더 중요한 경우 : 실제 긍정인 예측을 부정으로 잘못 판단하면 업무상 큰 영향을 끼치는 경우
- 정밀도가 더 중요한 경우 : 실제 부정인 예측을 긍정으로 잘못 판단하면 업무상 큰 영향을 끼치는 경우

In [1]:
# 오차 행렬 및 정밀도, 재현율을 모두 구하는 함수를 만들어보자.
# 사이킷런은 정밀도 계산을 위해 precision_score()를,
# 재현율 계산을 위해 recall_score()를 제공한다.

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred) :
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

In [10]:
# 타이타닉 생존자 예측 후 위 함수를 실행시켜 보자.
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator

# 이전에 타이타닉 전처리때 만든 함수들
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 분할
titanic_df = pd.read_csv('./data/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)


X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, \
                                                  test_size=0.2, random_state=0)


X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, \
                                                    test_size=0.20, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train , y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test , pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도 : 0.8492, 정밀도 : 0.7742, 재현율: 0.7869


C:\Users\korea_1\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### 정밀도/재현율 트레이드 오프

정밀도와 재현율은 상호 보완적인 평가지표이기 때문에 하나를 높이면 하나는 낮아지기 쉽다. 이런 사이를 trade off라고 한다. <br/><br/> 사이킷런의 분류 알고리즘은 예측 데이터가 특정 레이블에 속하는지를 계산하기 위해 먼저 개별 레이블별로 결정 확률을 구한다. 그리고 예측 확률이 큰 레이블값으로 예측하게 된다. 사이킷런은 개별 데이터별로 예측 확률을 반환하는 predict_proba()를 제공한다. 테스트 피처 데이터 세트를 파라미터로 입력하면 테스트 피처 레코드의 개별 클래스 예측 확률을 반환한다.

In [11]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print('pred_proba() 결과 shape : {0}'.format(pred_proba.shape))
print('pred_proba() array에서 앞 3개만 샘플로 추출 \n :', pred_proba[:3])

# 예측 확률 array와 예측 결과값 array를 병합(concatenate)해 예측 확률과 결과 값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1, 1)], axis=1)
print('두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])
print('0번째 칼럼이 0확률 1번째 칼럼이 1확률')

pred_proba() 결과 shape : (179, 2)
pred_proba() array에서 앞 3개만 샘플로 추출 
 : [[0.46170212 0.53829788]
 [0.87864222 0.12135778]
 [0.87728507 0.12271493]]
두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 
 [[0.46170212 0.53829788 1.        ]
 [0.87864222 0.12135778 0.        ]
 [0.87728507 0.12271493 0.        ]]
0번째 칼럼이 0확률 1번째 칼럼이 1확률


#### 정밀도와 재현율의 맹점

Positive 예측의 임곗값을 변경함에 따라 정밀도와 재현율의 수치가 변경된다. 임곗값의 이러한 변경은 업무 환경에 맞게 두 개의 수치를 상호 보완할 수 있는 수준에서 적용되어야 한다. 하나만 극단적으로 높이면 안된다.

##### 정밀도 100%가 되는 방법

확실한 기준이 되는 경우만 긍정 예측, 나머지는 부정 예측 1000명 중 진짜 확실한 1명만 암환자 긍정 예측을 하고 나머진 다 부정 예측 시 100%

##### 재현율 100%가 되는 방법

모든 환자를 긍정 예측

### 4. F1 스코어

![f1score](./images/f1score.png)

F1 스코어는 정밀도와 재현율을 결합한 지표이다. 정밀도와 재현율이 어느 한 쪽으로 치우치지 않는 수치를 나타낼 때 상대적으로 높은 값을 가집니다.<br/> 사이킷런에서도 이를 위한 API를 제공한다.
```python
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)
print('F1 스코어 : {0:.4f}'.format(f1))
```

### 5. ROC 곡선과 AUC

![roc](./images/ROC.png)

ROC 곡선과 이에 기반한 AUC 스코어는 이진 분류의 예측 성능 측정에서 중요하게 사용되는 지표이다. ROC 곡선은 우리말로 수신자 판단 곡선으로 불린다. <br/> ROC 곡선은 FPR(False Positive Rate)가 변할 때 TPR(True Positive Rate)가 어떻게 변하는지를 나타내는 곡선이다. FPR을  X축으로 TPR을 Y축으로 잡으면 FPR의 변화에 따른 TPR의 변화가 곡선 형태로 나타난다. <br/>TPR은 곧 재현율이다. 이는 곧 민감도로도 불리며 이에 대응하는 지표로 TNR(True Negative Rate)라고 불리는 특이성이 있다. 특이성은 다음과 같이 구할 수 있다. <br/><br/>FPR = FP / (FP + TN) = 1 - TNR = 1 - 특이성 <br/><br/> ROC 곡선은 직선에 가까울수록 성능이 떨어지는 것이며 멀어질수록 성능이 뛰어난 것이다. <br/> ROC 곡선은 FPR을 0부터 1까지 변경하면서 TPR의 변화 값을 구한다. FPR을 0부터 1까지 변경하는 방법은 분류 결정 임곗값을 변경하는 것이다. (predict 에서의)